In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем классы BaseEstimator и TransformerMixin, 
# позволяющие написать собственные классы
from sklearn.base import BaseEstimator, TransformerMixin
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс StandardScaler, позволяющий выполнить стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс OneHotEncoder, позволяющий выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score() для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс FunctionTransformer, позволяющий
# задавать пользовательские функции
from sklearn.preprocessing import FunctionTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс GridSearchCV, позволяющий 
# выполнить поиск по сетке
from sklearn.model_selection import GridSearchCV

In [2]:
# считываем данные
data = pd.read_csv('Data/cs-training.csv', index_col='Unnamed: 0')

In [3]:
# пишем функцию предварительной подготовки
def preprocessing(df):
    
    # значения переменной age меньше 18 заменяем
    # минимально допустимым значением возраста
    df['age'] = np.where(df['age'] < 18, 18, df['age'])
    
    # создаем переменную Ratio - отношение количества 
    # просрочек 90+ к общему количеству просрочек
    sum_of_delinq = (df['NumberOfTimes90DaysLate'] + 
                     df['NumberOfTime30-59DaysPastDueNotWorse'] + 
                     df['NumberOfTime60-89DaysPastDueNotWorse'])

    cond = (df['NumberOfTimes90DaysLate'] == 0) | (sum_of_delinq == 0)
    df['Ratio'] = np.where(
        cond, 0, df['NumberOfTimes90DaysLate'] / sum_of_delinq)
    
    # создаем индикатор нулевых значений переменной 
    # NumberOfOpenCreditLinesAndLoans
    df['NumberOfOpenCreditLinesAndLoans_is_0'] = np.where(
        df['NumberOfOpenCreditLinesAndLoans'] == 0, 'T', 'F')
    
    # создаем индикатор нулевых значений переменной 
    # NumberRealEstateLoansOrLines
    df['NumberRealEstateLoansOrLines_is_0'] = np.where(
        df['NumberRealEstateLoansOrLines'] == 0, 'T', 'F')
    
    # создаем индикатор нулевых значений переменной 
    # RevolvingUtilizationOfUnsecuredLines
    df['RevolvingUtilizationOfUnsecuredLines_is_0'] = np.where(
        df['RevolvingUtilizationOfUnsecuredLines'] == 0, 'T', 'F')
    
    # преобразовываем переменные в категориальные, применив
    # биннинг и перевод в единый строковый формат
    for col in ['NumberOfTime30-59DaysPastDueNotWorse', 
                'NumberOfTime60-89DaysPastDueNotWorse',
                'NumberOfTimes90DaysLate']:
        df.loc[df[col] > 3, col] = 4
        df[col] = df[col].apply(lambda x: f"cat_{x}")
        
    # создаем список списков - список 2-факторных взаимодействий
    lst = [
        ['NumberOfDependents', 
         'NumberOfTime30-59DaysPastDueNotWorse'],
        ['NumberOfTime60-89DaysPastDueNotWorse', 
         'NumberOfTimes90DaysLate'],
        ['NumberOfTime30-59DaysPastDueNotWorse', 
         'NumberOfTime60-89DaysPastDueNotWorse'],
        ['NumberRealEstateLoansOrLines_is_0', 
         'NumberOfTimes90DaysLate'],
        ['NumberOfOpenCreditLinesAndLoans_is_0', 
         'NumberOfTimes90DaysLate']
    ]
    
    # создаем взаимодействия
    for i in lst:
        f1 = i[0]
        f2 = i[1]
        df[f1 + ' + ' + f2 + '_interact'] = (df[f1].astype(str) + ' + ' 
                                             + df[f2].astype(str))

    # укрупняем редкие категории
    interact_columns = df.columns[df.columns.str.contains('interact')]
    for col in interact_columns:
        df.loc[df[col].value_counts()[df[col]].values < 55, col] = 'Other'
    
    return df

In [4]:
# применяем нашу функцию
data = preprocessing(data)

In [5]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
train, test, y_train, y_test = train_test_split(
    data.drop('SeriousDlqin2yrs', axis=1), 
    data['SeriousDlqin2yrs'], 
    test_size=.3, 
    stratify=data['SeriousDlqin2yrs'], 
    random_state=100)

In [6]:
# создаем собственный класс NumberOfDependentsReplacer, который
# заменяет пропуски переменной NumberOfDependents
# на определенное константное значение
class NumberOfDependentsReplacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        threshold: пороговое значение
        replace_value: значение, 
        на которое заменяем
    """
    def __init__(self, replace_value=0):
        self.replace_value = replace_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X.isnull(), self.replace_value, X)
        return X_replaced

In [7]:
# создаем собственный класс MonthlyIncomeReplacer, который
# заменяет пропуски и значения переменной MonthlyIncome
# ниже заданного порога на определенное константное значение
class MonthlyIncomeReplacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        threshold: пороговое значение
        replace_value: значение, 
        на которое заменяем
    """
    def __init__(self, threshold=1200, replace_value=1200):
        self.threshold = threshold
        self.replace_value = replace_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_trans = np.where((X.isnull()) | (X < self.threshold), 
                           self.replace_value, X)
        return X_trans

In [8]:
# создаем собственный класс UtilizationThresholdSetter, который
# заменяет значения переменной RevolvingUtilizationOfUnsecuredLines
# выше заданного порога на пропуски
class UtilizationThresholdSetter(BaseEstimator, TransformerMixin):
    """
    Параметры:
        threshold: пороговое значение
    """
    def __init__(self, threshold=2):
        self.threshold = threshold
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_trans = np.where(X > self.threshold, np.NaN, X)
        return X_trans

In [9]:
# создаем собственный класс, выполняющий биннинг
class CustomDiscretizer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        bins: список бинов.
    """
    def __init__(self, bins=np.arange(0, 1.05, 0.05)):
        self.bins = bins
    
    def fit(self, X, y=None):
        # fit опять бездельничает
        return self
    
    def transform(self, X):
        # transform выполняет всю работу: применяет преобразование 
        # с помощью заданного значения параметра bins
        self.bins[0] = -1
        X_bin = np.digitize(X, self.bins).astype('object')
        return X_bin

In [10]:
# создаем собственный класс, выполняющий винзоризацию
class OutlierRemover(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower_quantile: float, по умолчанию 0
        Нижний квантиль.
    upper_quantile: float, по умолчанию 0.75
        Верхний квантиль.
    k: float, по умолчанию 1.5
        Коэффициент.
    copy: bool, по умолчанию True
        Возвращает копию.
    """
    def __init__(self, copy=True, lower_quantile=0, 
                 upper_quantile=0.75, k=1.5):
        # все параметры для инициализации публичных атрибутов 
        # должны быть заданы в методе __init__
        
        # публичные атрибуты
        self.copy = copy
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.k = k
        
    def __is_numpy(self, X):
        # частный метод, который с помощью функции isinstance()
        # проверяет, является ли наш объект массивом NumPy
        return isinstance(X, np.ndarray)
                
    def fit(self, X, y=None):
        # fit должен принимать в качестве аргументов X и y
        
        # обучение модели осуществляется прямо здесь
        # создаем пустой словарь, в котором ключами
        # будут имена/целые числа, а значениями - кортежами
        self._dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                lower = np.quantile(X[:, col], self.lower_quantile)
                upper = np.quantile(X[:, col], self.upper_quantile)
                IQR = (upper - lower) * self.k
                self._dict[col] = (lower, upper, IQR)
        # в противном случае, т.е. если объект - датафрейм pandas,
        # выполняем следующие действия:
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # вычисляем и записываем в словарь
                lower = X[col].quantile(self.lower_quantile)
                upper = X[col].quantile(self.upper_quantile)
                IQR = (upper - lower) * self.k
                self._dict[col] = (lower, upper, IQR)

        # fit возвращает self
        return self
    
    def transform(self, X):
        # transform принимает в качестве аргумента только X
        
        # выполняем копирование массива во избежание 
        # предупреждения SettingWithCopyWarning
        # "A value is trying to be set on a copy of 
        # a slice from a DataFrame (Происходит попытка изменить 
        # значение в копии среза данных датафрейма)"
        if self.copy:
            X = X.copy()
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # применяем преобразование к X
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # заменяем
                X[:, col] = np.where(
                    X[:, col] < (self._dict[col][0] - self._dict[col][2]), 
                    self._dict[col][0] - self._dict[col][2], 
                    X[:, col])
                X[:, col] = np.where(
                    X[:, col] >= (self._dict[col][1] + self._dict[col][2]), 
                    self._dict[col][1] + self._dict[col][2], 
                    X[:, col])
                
        # в противном случае, т.е. если объект - датафрейм pandas,
        # выполняем следующие действия:
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # заменяем
                X[col] = np.where(
                    X[col] < (self._dict[col][0] - self._dict[col][2]), 
                    self._dict[col][0] - self._dict[col][2], 
                    X[col])
                X[col] = np.where(
                    X[col] >= (self._dict[col][1] + self._dict[col][2]), 
                    self._dict[col][1] + self._dict[col][2], 
                    X[col]) 
        # transform возвращает X
        return X

In [11]:
# создаем список категориальных переменных
cat_columns = train.dtypes[train.dtypes == 'object'].index.tolist()
# создаем список количественных переменных
num_columns = train.dtypes[train.dtypes != 'object'].index.tolist()
# создаем список с переменной NumberOfDependents
numberofdependents = ['NumberOfDependents']
# создаем список с переменной MonthlyIncome
income = ['MonthlyIncome']
# создаем список с переменной DebtRatio
debtratio = ['DebtRatio']
# создаем список с переменной RevolvingUtilizationOfUnsecuredLines
utilization = ['RevolvingUtilizationOfUnsecuredLines']
# удаляем из списка количественных переменных переменные MonthlyIncome,
# DebtRatio и RevolvingUtilizationOfUnsecuredLines
num_columns = list(set(num_columns).difference(
    set(numberofdependents + income + debtratio + utilization)))

In [12]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [13]:
# создаем конвейер для переменной NumberOfDependents
numberofdependents_pipe = Pipeline([
    ('trans', NumberOfDependentsReplacer()),
    ('scaler', StandardScaler())
])

In [14]:
# создаем конвейер для переменной MonthlyIncome
income_pipe = Pipeline([
    ('trans', MonthlyIncomeReplacer()),
    ('imp', SimpleImputer(strategy='median')),
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler())
])

In [15]:
# создаем конвейер для переменной DebtRatio
debtratio_pipe = Pipeline([
    ('outl', OutlierRemover()),
    ('scaler', StandardScaler())
])

In [16]:
# создаем конвейер для переменной DebtRatio
debtratio_pipe2 = Pipeline([
    ('outl', OutlierRemover()),
    ('binn', CustomDiscretizer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [17]:
# создаем конвейер для переменной RevolvingUtilizationOfUnsecuredLines
utilization_pipe = Pipeline([
    ('trans', UtilizationThresholdSetter()),
    ('imp', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [18]:
# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([('ohe', OneHotEncoder(sparse=False, 
                                           handle_unknown='ignore'))])

In [19]:
# создаем список трехэлементных кортежей, в котором первый
# элемент кортежа - название конвейера с преобразованиями
transformers = [('num', num_pipe, num_columns),
                ('numberofdependents', numberofdependents_pipe, numberofdependents),
                ('income', income_pipe, income),
                ('utilization', utilization_pipe, utilization),
                ('debtratio', debtratio_pipe, debtratio),
                ('debtratio2', debtratio_pipe2, debtratio),
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [20]:
# задаем итоговый конвейер
pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(C=0.03,
                                               solver='liblinear', 
                                               random_state=42))])

In [21]:
# задаем сетку гиперпараметров
param_grid = {'tf__utilization__trans__threshold': [1.5, 1.75, 2],
              'tf__numberofdependents__trans__replace_value': [0, 1, 2, 3],
              'tf__debtratio2__binn__bins': [np.arange(0, 1.05, 0.05), 
                                             np.arange(0, 1.05, 0.1)],
              'tf__income__trans__replace_value': [25000, 30000, 35000],
              'tf__debtratio__outl__upper_quantile': [0.75, 0.8, 0.85]
             }

In [22]:
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки
gs = GridSearchCV(pipe, 
                  param_grid, 
                  scoring='roc_auc',
                  cv=5,
                  n_jobs=-1)

In [23]:
# выполняем поиск по сетке
gs.fit(train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение AUC-ROC
print('Наилучшее значение AUC-ROC: {:.3f}'.format(gs.best_score_))
# смотрим значение AUC-ROC на тестовой выборке
print('AUC-ROC на тестовом наборе: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(test)[:, 1])))

Наилучшие значения гиперпараметров: {'tf__debtratio2__binn__bins': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), 'tf__debtratio__outl__upper_quantile': 0.8, 'tf__income__trans__replace_value': 35000, 'tf__numberofdependents__trans__replace_value': 3, 'tf__utilization__trans__threshold': 2}
Наилучшее значение AUC-ROC: 0.863
AUC-ROC на тестовом наборе: 0.864


In [24]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Data/cs-training.csv', index_col='Unnamed: 0')

In [25]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
fulldata = preprocessing(fulldata)

In [26]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('SeriousDlqin2yrs')

In [27]:
# записываем оптимальные значения гиперпараметров
best_params = gs.best_params_
# присваиваем итоговому конвейеру оптимальные 
# значения гиперпараметров
pipe.set_params(**best_params)
# обучаем итоговый конвейер с оптимальными значениями 
# гиперпараметров на всех исторических данных
pipe.fit(fulldata, y_fulldata)
# смотрим значение AUC-ROC
print('AUC-ROC на всей исторической выборке: {:.3f}'.format(
    roc_auc_score(y_fulldata, pipe.predict_proba(fulldata)[:, 1])))

AUC-ROC на всей исторической выборке: 0.864


In [28]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Data/cs-test.csv', 
                      index_col=0)
# записываем идентификатор набора новых данных
test_id = newdata.index

In [29]:
# выполняем предварительную обработку
# новых данных
newdata = preprocessing(newdata)

In [30]:
# при помощью итогового конвейера с оптимальными значениями 
# гиперпараметров, обученного на всей исторической выборке, 
# вычисляем вероятности для новых данных
prob = pipe.predict_proba(newdata)[:, 1]
# выведем вероятности для первых 5 наблюдений
prob[:5]

array([0.05477489, 0.05055286, 0.01712939, 0.09484548, 0.10469977])

In [31]:
pd.DataFrame({'Id': test_id, 'Probability': prob}).to_csv(
    'subm_giveme.csv', index=False)